In [ ]:
from functools import partial

import pandas as pd
import plotly.express as px
import sparql_dataframe

In [ ]:
Q_CESAR_TEMPLATE = """

        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xml: <http://www.w3.org/XML/1998/namespace>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX ecrm: <http://erlangen-crm.org/current/>
        PREFIX crmpc: <http://dramabase/ontology/crm_property_classes/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX efrbroo: <http://erlangen-crm.org/efrbroo/>
        PREFIX dram: <http://dramabase/ontology/>
        PREFIX dc: <http://purl.org/dc/elements/1.1/>
          PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>



SELECT ?dates (count(?dates) as ?oc) (?parent_label as ?lieux)
                # (group_concat(distinct(?loc_name); separator = ", ") as ?lieux)
WHERE 

{{
                   ?performance dram:R66_included_performed_version_of <{piece_uri}> . 
                         ?performance a efrbroo:F31_Performance ;
                                                      ecrm:P4_has_time-span ?pts .
                            ?pts dram:P82a_begin_of_the_begin ?start .
  
                                    OPTIONAL {{
   ?performance ecrm:P7_took_place_at ?location .
   ?location  ecrm:P87_is_identified_by ?loc_appelation .
   ?loc_appelation rdfs:label ?loc_name .
    
   
    OPTIONAL {{
      
      
      ?location ecrm:P89_falls_within ?parent .
     ?parent ecrm:P87_is_identified_by ?parent_appelation .
      ?parent_appelation rdfs:label ?parent_label . 
    
    }} 
    
  }}
  
  BIND (YEAR( ?start) as ?dates)
                        
}}
GROUP BY ?dates ?parent  ?parent_label #?location
ORDER BY ?dates
"""

In [ ]:
Q_RCF_TEMPLATE = """
PREFIX rcf: <http://rcf.logilab.fr/>

SELECT (?annee as ?dates) (COUNT(?representation) as ?oc) ("Paris" as ?lieux) WHERE {{
    ?representation rcf:aPourPiece <{piece_uri}>.
    ?journee rcf:aPourRepresentation ?representation.
    ?journee rcf:aPourDate ?date.
    BIND(year(?date) as ?annee)
}} GROUP BY ?annee ORDER BY ?annee
"""

In [ ]:
Q_ONSTAGE_TEMPLATE = """
PREFIX onstage: <https://www.vondel.humanities.uva.nl/onstage/>
PREFIX sc: <https://schema.org/>

select ?dates (count(?dates) as ?oc) (?place as ?lieux) where {{
  
  
  SERVICE <https://api.lod.uba.uva.nl/datasets/CREATE/ONSTAGE/services/ONSTAGE/sparql> {{

  <{piece_uri}> sc:headline ?orig_title .

  ?play sc:isBasedOn  <https://www.vondel.humanities.uva.nl/onstage/plays/630> ;
         sc:headline ?translat_title .
  ?event sc:workPerformed  ?play .
  ?event ^sc:subEvent ?parentEvent .
  ?parentEvent sc:startDate ?date ;
               sc:location ?location .
  ?location sc:name ?locName ;
      		sc:addressLocality ?locPlace .

    BIND (YEAR( ?date) as ?dates)
      BIND (CONCAT( ?locPlace, " - ", ?locName ) as ?place)

  }}
}}
GROUP BY ?dates ?place
ORDER BY ?dates
"""

In [ ]:
def _get_data(endpoint, template, origin, piece_uri):
    if not piece_uri:
        return None
    import warnings
    warnings.simplefilter("ignore") # Requests to CESAR generates warning.
    q = template.format(piece_uri=piece_uri)
    d = sparql_dataframe.get(endpoint, q)
    d["origin"] = origin
    return d

In [ ]:
get_data_cesar = partial(_get_data, "https://cesar2.huma-num.fr/sparql/", Q_CESAR_TEMPLATE, "CÉSAR")
get_data_rcf = partial(_get_data, "https://rcf-sparql.demo.logilab.fr/sparql/", Q_RCF_TEMPLATE, "RCF")
get_data_onstage = partial(_get_data, "https://api.lod.uba.uva.nl/datasets/CREATE/ONSTAGE/services/ONSTAGE/sparql", Q_ONSTAGE_TEMPLATE, "ON_STAGE")
get_data = lambda cesar_uri, rcf_uri, on_stage_uri: pd.concat(filter(lambda x: x is not None, (get_data_cesar(cesar_uri), get_data_rcf(rcf_uri), get_data_onstage(on_stage_uri))))

In [ ]:
PIECES = pd.read_csv("../works_alignments/rcf_cesare_alignments.csv").set_index("name").fillna('')
PIECES = {k: v for k, v in PIECES.iterrows()}

In [ ]:
from ipywidgets import interact


@interact
def plot_data(piece=sorted(PIECES.keys())):
    df = get_data(PIECES[piece].cesare_uri, PIECES[piece].rcf_uri, PIECES[piece].onstage_uri)
    return px.scatter(df, x="dates", y="lieux", size="oc", color="origin", marginal_x="rug", marginal_y="histogram")